In [ ]:
%%writefile model.py

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import GRU, Dropout, Dense, LSTM, SimpleRNN
import streamlit as st
import numpy as np
import datetime
import yfinance as yf
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from currency import get_prediction
from sklearn.preprocessing import MinMaxScaler
import warnings


warnings.filterwarnings('ignore')
def predict():
    base = st.sidebar.selectbox('Choose currency pair',
                                ['EUR-USD', 'GBP-USD', 'USD-CAD'])
    model = st.sidebar.selectbox('Choose model',
                                ['RNN', 'LSTM', 'GRU', 'MLP', 'ANN'])

    input_open_price = st.sidebar.number_input('Enter Open Price')
    input_high_price = st.sidebar.number_input('Enter High Price')
    input_low_price = st.sidebar.number_input('Enter Low Price')
    input_close_price = st.sidebar.number_input('Enter Close Price')

    input_price = np.array([input_open_price, input_high_price,
                            input_low_price, input_close_price, 0])

    input_price = input_price.reshape(1, -1)

    scaler = MinMaxScaler()
    input_price = scaler.fit_transform(input_price)

    if '-' in base:
      base = base.replace('-','')

    symbol= f'{base}=X'
    if st.sidebar.button('Accept'):
        data = download_data(symbol)
        st.sidebar.success('Data Downloaded!')
        if 'RNN' in model:
          RNN_model(data, symbol, input_price)
        if 'LSTM' in model:
          LSTM_model(data, symbol, input_price)
        if 'GRU' in model:
          GRU_model(data, symbol, input_price)
        if 'MLP' in model:
          MLP_model(data, symbol, input_price)
        if 'ANN' in model:
          ANN_model(data, symbol, input_price)

@st.cache_resource
def download_data(symbol):
    df = yf.download(symbol, start='2014-01-01',
                    end=datetime.date.today(), progress=False)
    df = df.drop('Adj Close', axis=1)
    df = df.dropna()
    return df

def df_to_X_y(df):
  df_as_np = df.to_numpy()
  X = []
  y = []
  date = []
  for i in range(len(df_as_np) - 1):
    columns_to_get = ['Open', 'High', 'Low', 'Close', 'Volume']
    row = df.loc[df.index[i], columns_to_get]
    #print(row)
    X.append(row)
    y.append(df['Close'][i + 1])
    date.append(df.index[i + 1])
  return np.array(X), np.array(y), np.array(date)

def RNN_model(df,symbol, input_price):

    X, y, date = df_to_X_y(df)

    # Initialize the scaler
    scaler = MinMaxScaler()

    X_scaled = scaler.fit_transform(X)

    q_80 = int(len(date) * .8)

    dates_train, X_train, y_train = date[:q_80], X_scaled[:q_80], y[:q_80]

    dates_test, X_test, y_test = date[q_80:], X_scaled[q_80:], y[q_80:]

    rnn = Sequential()

    rnn.add(SimpleRNN(50, activation='relu',
                      input_shape=(X_train.shape[1], 1),))

    rnn.add(Dense(1))

    rnn.compile(optimizer='adam', loss='mse')

    rnn.fit(X_train, y_train, epochs=10, batch_size=32,
            validation_split=0.2)

    preds = rnn.predict(X_test)
    score = r2_score(y_test, preds) * 100
    st.header('Currency Rate Prediction')
    st.write(f'Model Accuracy: {round(score)}%')
    get_prediction(rnn, df, input_price)

    st.header('Visualize Price Prediction Till Now')
    plt.figure(figsize=(15, 6))
    plt.plot(dates_test[-100:], preds[-100:])
    plt.plot(dates_test[-100:], y_test[-100:])
    plt.legend(['Testing Predictions', 'Testing Observations'])
    st.pyplot(plt.gcf())

def LSTM_model(df, symbol, input_price):
    X, y, date = df_to_X_y(df)

    # Initialize the scaler
    scaler = MinMaxScaler()

    X_scaled = scaler.fit_transform(X)

    q_80 = int(len(date) * .8)

    dates_train, X_train, y_train = date[:q_80], X_scaled[:q_80], y[:q_80]

    dates_test, X_test, y_test = date[q_80:], X_scaled[q_80:], y[q_80:]

    lstm = Sequential([layers.Input((5, 1)),
                    layers.LSTM(64),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(1)])

    lstm.compile(loss='mse',
                  optimizer=Adam(learning_rate=0.001, clipvalue=1.0),
                  metrics=['mean_absolute_error'])

    # Train the model
    lstm.fit(X_train, y_train, epochs=10)

    preds = lstm.predict(X_test)
    score = r2_score(y_test, preds) * 100
    st.header('Currency Rate Prediction')
    st.write(f'Model Accuracy: {round(score)}%')
    get_prediction(lstm, df, input_price)

    st.header('Visualization test data')
    plt.figure(figsize=(15, 6))
    plt.plot(dates_test[-100:], preds[-100:])
    plt.plot(dates_test[-100:], y_test[-100:])
    plt.legend(['Testing Predictions', 'Testing Observations'])
    st.pyplot(plt.gcf())

def MLP_model(df, symbol, input_price):
    X, y, date = df_to_X_y(df)

    # Initialize the scaler
    scaler = MinMaxScaler()

    X_scaled = scaler.fit_transform(X)

    q_80 = int(len(date) * .8)

    dates_train, X_train, y_train = date[:q_80], X_scaled[:q_80], y[:q_80]

    dates_test, X_test, y_test = date[q_80:], X_scaled[q_80:], y[q_80:]

    mlp = Sequential()

    mlp.add(Dense(100, activation="relu", input_dim=X_train.shape[1]))
    mlp.add(Dense(1, activation="relu"))

    mlp.compile(loss='mse', optimizer="adam")

    mlp.fit(X_train, y_train, epochs=10 )

    preds = mlp.predict(X_test)
    score = r2_score(y_test, preds) * 100
    st.header('Currency Rate Prediction')
    st.write(f'Model Accuracy: {round(score)}%')

    get_prediction(mlp,df, input_price)

    st.header('Visualization test data')
    plt.figure(figsize=(15, 6))
    plt.plot(dates_test[-100:], preds[-100:])
    plt.plot(dates_test[-100:], y_test[-100:])
    plt.legend(['Testing Predictions', 'Testing Observations'])
    st.pyplot(plt.gcf())

def ANN_model(df, symbol, input_price):
    X, y, date = df_to_X_y(df)

    # Initialize the scaler
    scaler = MinMaxScaler()

    X_scaled = scaler.fit_transform(X)

    q_80 = int(len(date) * .8)

    dates_train, X_train, y_train = date[:q_80], X_scaled[:q_80], y[:q_80]

    dates_test, X_test, y_test = date[q_80:], X_scaled[q_80:], y[q_80:]

    ann = Sequential()

    ann.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))

    ann.add(Dense(32, activation='relu'))
    ann.add(Dropout(0.2))

    ann.add(Dense(1, activation='linear'))

    ann.compile(optimizer='adam', loss='mean_squared_error')

    ann.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    preds = ann.predict(X_test)
    score = r2_score(y_test, preds) * 100
    st.header('Currency Rate Prediction')
    st.write(f'Model Accuracy: {round(score)}%')

    get_prediction(ann,df, input_price)

    st.header('Visualization test data')
    plt.figure(figsize=(15, 6))
    plt.plot(dates_test[-100:], preds[-100:])
    plt.plot(dates_test[-100:], y_test[-100:])
    plt.legend(['Testing Predictions', 'Testing Observations'])
    st.pyplot(plt.gcf())

def GRU_model(df, symbol, input_price):
    X, y, date = df_to_X_y(df)

    # Initialize the scaler
    scaler = MinMaxScaler()

    X_scaled = scaler.fit_transform(X)

    q_80 = int(len(date) * .8)

    dates_train, X_train, y_train = date[:q_80], X_scaled[:q_80], y[:q_80]

    dates_test, X_test, y_test = date[q_80:], X_scaled[q_80:], y[q_80:]

    regressorGRU = Sequential()

    # First GRU layer
    regressorGRU.add(GRU(units=100, return_sequences=True,
                    input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.3))
    # Second GRU layer
    regressorGRU.add(GRU(units=80, return_sequences=True,
                    input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # Third GRU layer
    regressorGRU.add(GRU(units=50, return_sequences=True,
                    input_shape=(X_train.shape[1],1), activation='tanh'))
    regressorGRU.add(Dropout(0.1))
    # Fourth GRU layer
    regressorGRU.add(GRU(units=30, activation='tanh'))
    regressorGRU.add(Dropout(0.2))
    # The output layer
    regressorGRU.add(Dense(units=1))

    # Compiling the GRU
    regressorGRU.compile(optimizer='adam',loss='mean_squared_error')

    # Fitting to the training set
    regressorGRU.fit(X_train,y_train,epochs=10,batch_size=150)

    preds = regressorGRU.predict(X_test).flatten()
    score = r2_score(y_test, preds) * 100
    st.header('Currency Rate Prediction')
    st.write(f'Model Accuracy: {round(score)}%')
    get_prediction(regressorGRU, df, input_price)

    st.header('Visualization test data')
    plt.figure(figsize=(15, 6))
    plt.plot(dates_test[-100:], preds[-100:])
    plt.plot(dates_test[-100:], y_test[-100:])
    plt.legend(['Testing Predictions', 'Testing Observations'])
    st.pyplot(plt.gcf())



Overwriting model.py


In [ ]:
%%writefile currency.py
from sklearn.preprocessing import MinMaxScaler
import streamlit as st
import pandas as pd
import numpy as np

def df_to_X_y(df):
  df_as_np = df.to_numpy()
  X = []
  y = []
  date = []
  for i in range(len(df_as_np) - 1):
    columns_to_get = ['Open', 'High', 'Low', 'Close', 'Volume']
    row = df.loc[df.index[i], columns_to_get]
    #print(row)
    X.append(row)
    y.append(df['Close'][i + 1])
    date.append(df.index[i + 1])
  return np.array(X), np.array(y), np.array(date)

def get_prediction(model, df, input_price):

    X, y, date = df_to_X_y(df)

    # Initialize the scaler
    scaler = MinMaxScaler()

    X_scaled = scaler.fit_transform(X)

    q_80 = int(len(date) * .8)

    dates_train, X_train, y_train = date[:q_80], X_scaled[:q_80], y[:q_80]

    dates_test, X_test, y_test = date[q_80:], X_scaled[q_80:], y[q_80:]

    preds = model.predict(X_test)

    st.write("Predicted Price:", model.predict(input_price)[0, 0])

    # Combine the predicted values (y_pred) with the corresponding dates
    predictions_df = pd.DataFrame({'Date': dates_test,
                                  'Predicted': preds.flatten()})
    """
    # Get the result for the day before the last day
    day_before_last_prediction = predictions_df.iloc[-2]
    last_day_prediction = predictions_df.iloc[-1]

    # Access the date and predicted value for the day before the last day
    day_before_last_predicted_date = day_before_last_prediction['Date']
    day_before_last_predicted_value = day_before_last_prediction['Predicted']

    # Access the date and predicted value for the last day
    last_day_predicted_date = last_day_prediction['Date']
    last_day_predicted_value = last_day_prediction['Predicted']

    st.write("Day Before Last Predicted Date:", day_before_last_predicted_date)
    st.write("Corresponding Predicted Value (Day Before Last):",
            day_before_last_predicted_value)

    st.write("\nLast Day Predicted Date:", last_day_predicted_date)
    st.write("Corresponding Predicted Value (Last Day):",
            last_day_predicted_value)

    """

Overwriting currency.py


In [ ]:
%%writefile app.py

import streamlit as st
from model import predict

def main():
    st.sidebar.header('Price Predictor App')

    predict()

if __name__ == '__main__':
    main()

Overwriting app.py


In [ ]:
!pip install -q streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.884s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",
      urllib.request.urlopen('https://ipv4.icanhazip.com')
      .read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.141.215.27


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 3.419s
your url is: https://eleven-glasses-flow.loca.lt
/root/.npm/_npx/3861/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:40633 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/3861/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (events.js:315:20)
    at emitErrorNT (internal/streams/destroy.js:106:8)
    at emitErrorCloseNT (internal/streams/destroy.js:74:3)
    at processTicksAndRejections (internal/process/task_queues.js:80:21)
